<a href="https://colab.research.google.com/github/DJCordhose/practical-llm/blob/main/Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi Lingual Medical Assessment Classification

## LLama 3.1 8B
https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct

### Measurements

#### 4 Bit T4
* Negatives: 35 s
* Positives: 38 s

#### 8 Bit T4
* Negatives: 23 s
* Positives: 23 s

#### Full Res L4
* Negatives: 12.5 s
* Positives: 12.5 s

## Phi 3.5
* https://techcommunity.microsoft.com/t5/ai-azure-ai-services-blog/discover-the-new-multi-lingual-high-quality-phi-3-5-slms/ba-p/4225280
* https://huggingface.co/microsoft/Phi-3.5-mini-instruct
* https://huggingface.co/microsoft/Phi-3.5-MoE-instruct

### Measurements

#### 4 Bit A100
* Negatives: 50 s
* Positives: 50 s


In [1]:
%%time

!pip install --upgrade -q transformers accelerate flash_attn torch bitsandbytes

CPU times: user 13.1 ms, sys: 4.12 ms, total: 17.2 ms
Wall time: 2.86 s


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from IPython.display import Markdown

In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) 
Token is valid (permission: read).


In [5]:
!nvidia-smi

Fri Aug 23 11:48:49 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   65C    P8              18W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [25]:
# kind = 'Lllama_3.1_8B_4bit'
# kind = 'Lllama_3.1_8B_8bit'
kind = 'Lllama_3.1_8B_16bit'
# kind = 'Phi-3.5-MoE_4bit'
kind = "Phi-3.5-mini_16bit"

# lang = "de"
lang = "en"

In [7]:
if "Lllama_3.1_8B" in kind:
  model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
elif "Phi-3.5-MoE" in kind:
  model_id = "microsoft/Phi-3.5-MoE-instruct"
else:
  model_id = "microsoft/Phi-3.5-mini-instruct"

print(model_id)

microsoft/Phi-3.5-mini-instruct


In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

***note:*** execute in a termial 'watch -n 0.5 nvidia-smi' to see the GPU usage and when the model is loaded onto it

In [9]:
%%time

from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import torch

torch_dtype = None
quantization_config = None


if "8bit" in kind:
  print("Using 8Bit quantization")
  quantization_config = BitsAndBytesConfig(load_in_8bit=True)
elif "4bit" in kind:
  print("Using 4Bit quantization")
  quantization_config = BitsAndBytesConfig(load_in_4bit=True)
else:
  print("Using Full Resolution")
  torch_dtype = torch.bfloat16

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    torch_dtype=torch_dtype,
    device_map="cuda",
    trust_remote_code=True
)

Using Full Resolution


config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

CPU times: user 10.7 s, sys: 11.6 s, total: 22.4 s
Wall time: 38.7 s


In [10]:
!nvidia-smi

Fri Aug 23 11:49:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   64C    P0              32W /  72W |   7481MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [26]:
positive_en = [
  "With the diagnosis named here, the need for compensation to ensure the basic need is conceivable.",
  "The socio-medical prerequisites for the prescribed aid supply have been met.",
  "Everyday relevant usage benefits have been determined.",
  "Socio-medical indication for the aid is confirmed.",
  "Contraindications have been excluded; there are no contraindications for the use of the requested aid."
]

In [27]:
negative_en = [
  "No specific findings can be derived from the diagnosis currently named as the basis for the regulation.",
  "According to the service extracts from the health insurance, the insured has already been provided with the functional product requested according to its area of application.",
  "A medically comprehensible explanation as to why the use of an orthopedic aid corresponding to the findings is not sufficient and instead electric foot lifter stimulation for walking would be more appropriate and therefore necessary has not been transmitted.",
  "From an overall view of the information available here, it cannot be seen how the supply of the insured with the product could be justified, nor can the safety of such a supply be confirmed.",
  "A medical justification for why a product not listed in the directory of aids should be used in the present case has not been transmitted."
]

In [28]:
positive_de = [
  "Bei der hier benannten Diagnose ist das Erfordernis eines Ausgleichs zur Sicherstellung des Grundbedürfnisses denkbar.",
  "Die sozialmedizinischen Voraussetzungen für die verordnete Hilfsmittelversorgung sind erfüllt.",
  "Alltagsrelevante Gebrauchsvorteile werden festgestellt.",
  "Sozialmedizinische Indikation für das Hilfsmittel wird bestätigt.",
  "Kontraindikationen wurden ausgeschlossen, es liegen keine Gegenanzeigen für die Verwendung des beantragten Hilfsmittels vor."
]

In [29]:
negative_de = [
  "Aus der aktuell als verordnungsbegründend benannten Diagnose lässt sich kein konkreter Befund ableiten.",
  "Gemäß den Leistungsauszügen der Krankenkasse ist der Versicherte bereits entsprechend dem Einsatzbereich des beantragten funktionellen Produkt versorgt.",
  "Eine medizinisch nachvollziehbare Begründung, weshalb der Einsatz einer befundadäquaten orthopädietechnischen Hilfsmittelversorgung nicht ausreichend und stattdessen eine elektrische Fußheberstimulation zum Gehen zweckmäßiger und deshalb notwendig wäre, wurde nicht übermittelt.",
  "In der Gesamtschau der hier vorliegenden Informationen kann nicht erkannt werden, wie die Versorgung des Versicherten mit dem Produkt begründet werden könnte, noch kann die Unbedenklichkeit einer solchen Versorgung bestätigt werden.",
  "Eine ärztliche Begründung, warum im vorliegenden Fall ein nicht im Hilfsmittelverzeichnis gelistetes Produkt zum Einsatz kommen soll, wird nicht übermittelt."
]

In [30]:

if lang == "de":
  negative = negative_de
  positive = positive_de
else:
  negative = negative_en
  positive = positive_en



In [31]:
assessment = negative[0]
# assessment = positive[0]

In [32]:
%%time

if lang == "de":
  messages = [
    {"role": "system", "content": "Du bist ein kompetenter Experte auf dem Gebiet der gesetzlichen Krankenversicherung und sprichst Deutsch. Antworte präzise, ernst und formell."},
    {"role": "user", "content": f'''
    Was ist das Ergebnis der Bewertung? Wird eine positive oder negative Empfehlung gegeben? Antworte mit 'Ja' oder 'Nein' und gib anschließend eine sehr kurze Begründung für die Einschätzung."

    # Assessment
    {assessment}

    '''}
  ]
else:
  messages = [
    {"role": "system", "content": "You are an English-speaking, competent expert in the field of statutory health insurance. Answer consice, serious and formal."},
    {"role": "user", "content": f'''
What is the result of the assessment? Is a positive or negative recommendation given? Answer with "Yes" or "No" and then provide a brief justification for your assessment.

# Assessment
{assessment}

'''}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=False
)
response = outputs[0][input_ids.shape[-1]:]
Markdown(tokenizer.decode(response, skip_special_tokens=True))

CPU times: user 4.71 s, sys: 6.27 ms, total: 4.72 s
Wall time: 4.71 s


Nein, die Bewertung gibt keine positive oder negative Empfehlung ab. Begründung: Die Aussage konzentriert sich auf die fehlende Ableitung eines konkreten Befunds aus der Diagnose und bewertet somit neutral, ohne eine Empfehlung zu geben.

In [33]:
def eval_assessment(assessment):
  if lang == "de":
    yes = "Ja"
    no = "Nein"
    messages = [
  {"role": "system", "content": "Du bist ein kompetenter Experte auf dem Gebiet der gesetzlichen Krankenversicherung und sprichst Deutsch. Antworte präzise, ernst und formell."},
  {"role": "user", "content": f'''
  Was ist das Ergebnis der Bewertung? Wird eine positive oder negative Empfehlung gegeben? Antworte mit 'Ja' oder 'Nein' und gib anschließend eine sehr kurze Begründung für die Einschätzung."

  # Assessment
  {assessment}

  '''}
  ]
  else:
    yes = "Yes"
    no = "No"
    messages = [
      {"role": "system", "content": "You are an English-speaking, competent expert in the field of statutory health insurance. Answer consice, serious and formal."},
      {"role": "user", "content": f'''
  What is the result of the assessment? Is a positive or negative recommendation given? Answer with "Yes" or "No" and then provide a brief justification for your assessment.

  # Assessment
  {assessment}

  '''}
  ]

  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=512,
      eos_token_id=terminators,
      pad_token_id=tokenizer.eos_token_id,
      do_sample=False
  )
  response = outputs[0][input_ids.shape[-1]:]
  result = tokenizer.decode(response, skip_special_tokens=True)
  if result.startswith(yes):
    return "Positive", result
  elif result.startswith(no):
    return "Negative", result
  else:
    return "Neutral", result

## Negative

In [34]:
%%time

negative_results = []
negative_explanations = []

for assessment in negative:
  print(f"Assessment: {assessment}")
  result, explanation = eval_assessment(assessment)
  negative_results.append(result)
  negative_explanations.append(explanation)
  print(f"{result}: {explanation}")
  print("-----")

Assessment: Aus der aktuell als verordnungsbegründend benannten Diagnose lässt sich kein konkreter Befund ableiten.
Negative: Nein, eine negative Empfehlung wird gegeben. Die Begründung ist, dass die Diagnose nicht konkrete Befunde liefert, was auf einen Mangel an ausreichenden Informationen für eine positive Bewertung hinweist.
-----
Assessment: Gemäß den Leistungsauszügen der Krankenkasse ist der Versicherte bereits entsprechend dem Einsatzbereich des beantragten funktionellen Produkt versorgt.
Positive: Ja, eine positive Empfehlung wird gegeben. Die Begründung ist, dass der Versicherte gemäß den Leistungsauszügen der Krankenkasse bereits die notwendige Versorgung für das funktionelle Produkt erhalten hat, was auf eine effektive und zufriedenstellende Versorgung hinweist.
-----
Assessment: Eine medizinisch nachvollziehbare Begründung, weshalb der Einsatz einer befundadäquaten orthopädietechnischen Hilfsmittelversorgung nicht ausreichend und stattdessen eine elektrische Fußheberstimul

## Positive

In [35]:
%%time

positive_results = []
positive_explanations = []

for assessment in positive:
  print(f"Assessment: {assessment}")
  result, explanation = eval_assessment(assessment)
  positive_results.append(result)
  positive_explanations.append(explanation)
  print(f"{result}: {explanation}")
  print("-----")

Assessment: Bei der hier benannten Diagnose ist das Erfordernis eines Ausgleichs zur Sicherstellung des Grundbedürfnisses denkbar.
Positive: Ja, eine positive Empfehlung wird gegeben. Die Begründung ist, dass die Aussage anerkennt das Potenzial für einen Ausgleich, um das Grundbedürfnis zu erfüllen, was auf eine proaktive und problembewusste Herangehensweise an die Versorgung hinweist.
-----
Assessment: Die sozialmedizinischen Voraussetzungen für die verordnete Hilfsmittelversorgung sind erfüllt.
Positive: Ja, eine positive Empfehlung wird gegeben. Die Begründung ist, dass die sozialmedizinischen Voraussetzungen für die Hilfsmittelversorgung erfüllt sind, was darauf hindeutet, dass der Antragsteller für die gewünschte Versorgung berechtigt ist.
-----
Assessment: Alltagsrelevante Gebrauchsvorteile werden festgestellt.
Positive: Ja, eine positive Empfehlung. Die Begründung ist, dass die Identifizierung von alltagsrelevanten Gebrauchsvorteilen auf eine erfolgreiche Bewertung hinweist, die

In [36]:
!nvidia-smi

Fri Aug 23 11:52:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   74C    P0              61W /  72W |   7809MiB / 23034MiB |     47%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [37]:
import pandas as pd

df = pd.DataFrame({
    'assesment': negative + positive,
    'y_true': ['Negative'] * len(negative) + ['Positive'] * len(positive),
    'y_hat': negative_results + positive_results,
    'explanation': negative_explanations + positive_explanations
})
df

,assesment,y_true,y_hat,explanation
0,Aus der aktuell als verordnungsbegründend bena...,Negative,Negative,"Nein, eine negative Empfehlung wird gegeben. D..."
1,Gemäß den Leistungsauszügen der Krankenkasse i...,Negative,Positive,"Ja, eine positive Empfehlung wird gegeben. Die..."
2,"Eine medizinisch nachvollziehbare Begründung, ...",Negative,Negative,"Nein, eine positive Empfehlung wird nicht gege..."
3,In der Gesamtschau der hier vorliegenden Infor...,Negative,Negative,"Nein, die Bewertung gibt keine positive Empfeh..."
4,"Eine ärztliche Begründung, warum im vorliegend...",Negative,Negative,"Nein, die Bewertung deutet auf eine negative E..."
5,Bei der hier benannten Diagnose ist das Erford...,Positive,Positive,"Ja, eine positive Empfehlung wird gegeben. Die..."
6,Die sozialmedizinischen Voraussetzungen für di...,Positive,Positive,"Ja, eine positive Empfehlung wird gegeben. Die..."
7,Alltagsrelevante Gebrauchsvorteile werden fest...,Positive,Positive,"Ja, eine positive Empfehlung. Die Begründung i..."
8,Sozialmedizinische Indikation für das Hilfsmit...,Positive,Positive,"Ja, eine positive Empfehlung wird gegeben. Die..."
9,"Kontraindikationen wurden ausgeschlossen, es l...",Positive,Positive,"Ja, eine positive Empfehlung wird gegeben. Die..."


In [38]:
df.to_excel(f'results_{kind}_{lang}.xlsx', index=False)

In [39]:
!ls -l

total 68
-rw-r--r-- 1 root root 6332 Aug 23 11:41 results_Lllama_3.1_8B_16bit_de.xlsx
-rw-r--r-- 1 root root 6318 Aug 23 11:39 results_Lllama_3.1_8B_16bit_en.xlsx
-rw-r--r-- 1 root root 6198 Aug 23 11:44 results_Lllama_3.1_8B_4bit_de.xlsx
-rw-r--r-- 1 root root 6361 Aug 23 11:46 results_Lllama_3.1_8B_4bit_en.xlsx
-rw-r--r-- 1 root root 6290 Aug 23 11:28 results_Lllama_3.1_8B_8bit_de.xlsx
-rw-r--r-- 1 root root 6372 Aug 23 11:32 results_Lllama_3.1_8B_8bit_en.xlsx
-rw-r--r-- 1 root root 6590 Aug 23 11:52 results_Phi-3.5-mini_16bit_de.xlsx
-rw-r--r-- 1 root root 6697 Aug 23 11:50 results_Phi-3.5-mini_16bit_en.xlsx
drwxr-xr-x 1 root root 4096 Aug 21 13:28 sample_data
